In [2]:
import pandas as pd
import ast
from IPython.display import display
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np

In [3]:
class CFG:
    dataset_dir="../input/ranzcr-clip-catheter-line-classification/"

In [4]:
annotations=pd.read_csv(f"{CFG.dataset_dir}train_annotations.csv")
annotatin_cvc=annotations[annotations["label"].str.contains("CVC")]
print(len(annotatin_cvc))

11629


In [5]:
def visualize_annotations(uid,annot_data):
    fig,ax=plt.subplots(1,1,figsize=(4,4))
    image = cv2.imread(f"{CFG.dataset_dir}train/{uid}.jpg")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    ax.imshow(image)
    ax.scatter(annot_data[:-2,0],annot_data[:-2,1],c="cyan")
    ax.scatter(annot_data[-1,0],annot_data[-1,1],c="red")
    ax.axis("off")
    fig.savefig(f"../input/cvc_annotated/{uid}.jpg")

In [6]:
for idx,row in annotatin_cvc.iterrows():
    annot_data=ast.literal_eval(row["data"])
    annot_data=np.array(annot_data)
    visualize_annotations(row["StudyInstanceUID"],annot_data)

<ipython-input-5-673108123905>:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig,ax=plt.subplots(1,1,figsize=(4,4))


KeyboardInterrupt: 